In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

## Imports

In [2]:
from pathlib import Path
import shutil
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.model import make_model, freeze_all_vgg, unfreeze_last_vgg
from utils.data import train_test_valid_split, filter_binary_labels, optimize_dataset, create_split, delete_folder

## Image files management -> train, test, valid splits

In [4]:
DATASET_SOURCE_PATH = Path(r'../data/dataset')
SPLITS_DESTINATION_PATH = Path(r'../data')
UNDERSAMPLE_RATIO = 0.8  # controls the under sampling for majority class. Use 'None' to disable
TEST_SIZE = 0.15
VALID_SIZE = 0.15

X_train, X_test, X_valid = train_test_valid_split(DATASET_SOURCE_PATH, test_size=TEST_SIZE, valid_size=VALID_SIZE, undersample_ratio=UNDERSAMPLE_RATIO)

splits = [('train', X_train), ('test', X_test), ('valid', X_valid)]

for split in splits:
    destination_path = Path(SPLITS_DESTINATION_PATH) / split[0]
    delete_folder(destination_path)
    create_split(split[1], destination_path)

--------------DELETE TRAIN SPLIT------------
--------------COPY TRAIN SPLIT------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 781/781 [00:00<00:00, 827.33it/s]


--------------DELETE TEST SPLIT------------
--------------COPY TEST SPLIT------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 177/177 [00:00<00:00, 800.90it/s]


--------------DELETE VALID SPLIT------------
--------------COPY VALID SPLIT------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [00:00<00:00, 819.51it/s]


## Dataset loading

In [5]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
SEED = None

train_path = SPLITS_DESTINATION_PATH / 'train'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_path, image_size=(IMG_HEIGHT, IMG_WIDTH),\
                                                               batch_size=BATCH_SIZE, shuffle=True, \
                                                               label_mode='categorical', seed=SEED)

valid_path = SPLITS_DESTINATION_PATH / 'valid'
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(valid_path, image_size=(IMG_HEIGHT, IMG_WIDTH),\
                                                               batch_size=BATCH_SIZE, shuffle=True, \
                                                               label_mode='categorical', seed=SEED)

class_names = train_ds.class_names
assert class_names == valid_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE

if len(class_names) == 2:  # take the one-hot-encoded matrix of labels and convert to a vector if binary classification
    train_ds = train_ds.map(filter_binary_labels, num_parallel_calls=AUTOTUNE)
    valid_ds = valid_ds.map(filter_binary_labels, num_parallel_calls=AUTOTUNE)
train_ds = optimize_dataset(train_ds)
valid_ds = optimize_dataset(valid_ds)

Found 780 files belonging to 2 classes.
Found 168 files belonging to 2 classes.


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label_idx = labels.numpy()[i][0] if len(class_names) == 2 else np.argmax(labels.numpy()[i], axis=0)
        plt.title(class_names[label_idx])
        plt.axis("off")

## Model

In [7]:
N_HIDDEN = 512
BASE_LR = 0.001

model = make_model(n_classes=len(class_names), n_hidden=N_HIDDEN, img_height=IMG_HEIGHT, img_width=IMG_WIDTH)
freeze_all_vgg(model)

loss = tf.keras.losses.CategoricalCrossentropy() if len(class_names) > 2 else tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=BASE_LR),
              loss=loss, metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                             

#### Classifier initial training

In [7]:
# TODO - delete last trained model

In [8]:
LOG_PATH = Path(r'../models/vgg16/logs')
CHECKPOINTS_PATH = Path(r'../models/vgg16/checkpoints')
BASE_EPOCHS = 30

tb = TensorBoard(log_dir=LOG_PATH)
checkpoint = ModelCheckpoint(CHECKPOINTS_PATH / 'train_{epoch}.tf', verbose=1, save_weights_only=True,
                             save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1)

history = model.fit(train_ds, epochs=BASE_EPOCHS, validation_data=valid_ds, callbacks=[tb, checkpoint, reduce_lr,
                                                                                       early_stopping])

Epoch 1/30
13/13 [==============================] - ETA: 0s - loss: 0.6858 - accuracy: 0.6885
Epoch 00001: val_loss improved from inf to 2.91570, saving model to ..\models\vgg16\checkpoints\train_1.tf
13/13 [==============================] - 44s 2s/step - loss: 0.6858 - accuracy: 0.6885 - val_loss: 2.9157 - val_accuracy: 0.5952 - lr: 0.0010
Epoch 2/30
13/13 [==============================] - ETA: 0s - loss: 0.4932 - accuracy: 0.7846
Epoch 00002: val_loss improved from 2.91570 to 1.60101, saving model to ..\models\vgg16\checkpoints\train_2.tf
13/13 [==============================] - 7s 518ms/step - loss: 0.4932 - accuracy: 0.7846 - val_loss: 1.6010 - val_accuracy: 0.6488 - lr: 0.0010
Epoch 3/30
13/13 [==============================] - ETA: 0s - loss: 0.4585 - accuracy: 0.8205
Epoch 00003: val_loss improved from 1.60101 to 0.80197, saving model to ..\models\vgg16\checkpoints\train_3.tf
13/13 [==============================] - 7s 519ms/step - loss: 0.4585 - accuracy: 0.8205 - val_loss: 0.

#### Fine tuning

In [9]:
FINE_TUNE_AT_LAYER = 15
FINE_TUNING_EPOCHS = 30
FINE_TUNING_LR = 0.001
FINAL_MODEL_NAME = 'trained_weights'
FINAL_MODEL_SAVE_PATH = CHECKPOINTS_PATH / FINAL_MODEL_NAME

unfreeze_last_vgg(model, which_freeze=FINE_TUNE_AT_LAYER)

total_epochs = BASE_EPOCHS + FINE_TUNING_EPOCHS
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNING_LR),
              loss=loss, metrics=['accuracy'])
history = model.fit(train_ds, epochs=total_epochs, validation_data=valid_ds, callbacks=[tb, checkpoint, reduce_lr, early_stopping], \
                    initial_epoch=history.epoch[-1])

model.save(FINAL_MODEL_SAVE_PATH, save_format='h5')

Epoch 25/60
13/13 [==============================] - ETA: 0s - loss: 0.7369 - accuracy: 0.6769
Epoch 00025: val_loss did not improve from 0.58987
13/13 [==============================] - 12s 583ms/step - loss: 0.7369 - accuracy: 0.6769 - val_loss: 1.4477 - val_accuracy: 0.6786 - lr: 0.0010
Epoch 26/60
13/13 [==============================] - ETA: 0s - loss: 0.7996 - accuracy: 0.6026
Epoch 00026: val_loss did not improve from 0.58987
13/13 [==============================] - 7s 555ms/step - loss: 0.7996 - accuracy: 0.6026 - val_loss: 9.1605 - val_accuracy: 0.4405 - lr: 0.0010
Epoch 27/60
13/13 [==============================] - ETA: 0s - loss: 0.8180 - accuracy: 0.5641
Epoch 00027: val_loss did not improve from 0.58987
13/13 [==============================] - 7s 557ms/step - loss: 0.8180 - accuracy: 0.5641 - val_loss: 0.6965 - val_accuracy: 0.5536 - lr: 0.0010
Epoch 28/60
13/13 [==============================] - ETA: 0s - loss: 0.6974 - accuracy: 0.6192
Epoch 00028: val_loss did not impr

Epoch 52/60
13/13 [==============================] - ETA: 0s - loss: 0.4431 - accuracy: 0.7936
Epoch 00052: val_loss improved from 0.53364 to 0.52467, saving model to ..\models\vgg16\checkpoints\train_52.tf
13/13 [==============================] - 8s 617ms/step - loss: 0.4431 - accuracy: 0.7936 - val_loss: 0.5247 - val_accuracy: 0.7500 - lr: 1.0000e-06
Epoch 53/60
13/13 [==============================] - ETA: 0s - loss: 0.4566 - accuracy: 0.7936
Epoch 00053: val_loss improved from 0.52467 to 0.51661, saving model to ..\models\vgg16\checkpoints\train_53.tf
13/13 [==============================] - 8s 597ms/step - loss: 0.4566 - accuracy: 0.7936 - val_loss: 0.5166 - val_accuracy: 0.7500 - lr: 1.0000e-06
Epoch 54/60
13/13 [==============================] - ETA: 0s - loss: 0.4621 - accuracy: 0.7795
Epoch 00054: val_loss improved from 0.51661 to 0.51177, saving model to ..\models\vgg16\checkpoints\train_54.tf
13/13 [==============================] - 8s 595ms/step - loss: 0.4621 - accuracy: 0

#### Model evaluation

In [10]:
# TODO - load model if not trained

In [13]:
test_path = Path(r'../data/test')
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_path, image_size=(IMG_HEIGHT, IMG_WIDTH), \
                                                              batch_size=BATCH_SIZE, shuffle=False, \
                                                              label_mode='categorical')
assert class_names == test_ds.class_names

if len(class_names) == 2:  # take the one-hot-encoded matrix of labels and convert to a vector if binary classification
    test_ds = test_ds.map(filter_binary_labels, num_parallel_calls=AUTOTUNE)
test_ds = optimize_dataset(test_ds)

metrics = model.evaluate(test_ds)
print('Loss: {} --------- Accuracy: {}%'.format(metrics[0], np.round(metrics[1]*100, 2)))

y_pred = model.predict(test_ds)
y_true = tf.concat([y for x, y in test_ds], axis=0)
if len(class_names) == 2: # uses a threshold for the predictions if binary classification problem
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    y_true = y_true.numpy()
else: # uses argmax if not binary classification
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_true.numpy(), axis=1)

print(classification_report(y_true, y_pred, target_names=class_names, digits=2))

pred_labels = [('PRED_' + class_name) for class_name in class_names]
real_labels = [('REAL_' + class_name) for class_name in class_names]
pd.DataFrame(confusion_matrix(y_true, y_pred), columns=pred_labels, index=real_labels)

Found 177 files belonging to 2 classes.
3/3 [==============================] - 14s 7s/step - loss: 0.3680 - accuracy: 0.8701
Loss: 0.36798617243766785 --------- Accuracy: 87.01%
              precision    recall  f1-score   support

           0       0.95      0.81      0.88       102
           1       0.79      0.95      0.86        75

    accuracy                           0.87       177
   macro avg       0.87      0.88      0.87       177
weighted avg       0.88      0.87      0.87       177



,PRED_0,PRED_1
REAL_0,83,19
REAL_1,4,71
